### Naive Bayse
- 각 변수들이 서로 독립하다는 순진한 가정. 범주형일 때 좋은 선능

아래 수식에서 제시된 바와 같이 각 변수들이 서로 독립이라는 순진한 가정하에서  P(xi|y) 를 계산하므로 각 변수들( xi )이 범주형일 때 좋은 성능을 갖는다.

posteriorP(y|X)=likelihood∗priormarginalprobability=P(X|y)P(y)/P(X)=P(x1,⋯,xp|y)P(y)/P(x1,⋯,xp)=P(x1|y)⋯P(xp|y)P(y)P(x1)⋯P(xp)=∏pi=1P(xi|y)P(y)∏pi=1P(xi) 

P(y=1|X)logP(y=1|X)P(y=0|X)logP(y=0|X)=P(X|y=1)P(y=1)/P(X)=∏pi=1P(xi|y=1)P(y=1)∏pi=1P(xi)∼∏i=1pP(xi|y=1)P(y=1)∼∑i=1plogP(xi|y=1)+logP(y=1)=P(X|y=0)P(y=0)/P(X)=∏pi=1P(xi|y=0)P(y=1)∏pi=1P(xi)∼∏i=1pP(xi|y=0)P(y=0)∼∑i=1plogP(xi|y=0)+logP(y=0) 

P(x1,…,xp) 는 관측된 상수이므로 다음과 같은 분류 규칙을 사용할 수 있다:
P(y∣x1,…,xp)∝P(y)∏i=1pP(xi∣y)⇓y^=argmaxyP(y)∏i=1pP(xi∣y)

In [35]:
import pandas as pd
import numpy as np
data = pd.read_csv('weather.nominal.csv')
data

,outlook,temperature,humidity,windy,play
0,overcast,hot,high,False,yes
1,overcast,cool,normal,True,yes
2,overcast,mild,high,True,yes
3,overcast,hot,normal,False,yes
4,rainy,mild,high,False,yes
5,rainy,cool,normal,False,yes
6,rainy,cool,normal,True,no
7,rainy,mild,normal,False,yes
8,rainy,mild,high,True,no
9,sunny,hot,high,False,no


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   outlook       14 non-null     object
 1    temperature  14 non-null     object
 2    humidity     14 non-null     object
 3    windy        14 non-null     bool  
 4    play         14 non-null     object
dtypes: bool(1), object(4)
memory usage: 590.0+ bytes


In [65]:
data.columns = data.columns.str.strip()   ## 칼럼 양옆 공백 제거.(정렬)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   outlook      14 non-null     object
 1   temperature  14 non-null     object
 2   humidity     14 non-null     object
 3   windy        14 non-null     bool  
 4   play         14 non-null     object
dtypes: bool(1), object(4)
memory usage: 590.0+ bytes


In [66]:
data.windy = data.windy.astype(str)  # 타입 정렬.
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   outlook      14 non-null     object
 1   temperature  14 non-null     object
 2   humidity     14 non-null     object
 3   windy        14 non-null     object
 4   play         14 non-null     object
dtypes: object(5)
memory usage: 688.0+ bytes


In [91]:
tabs = []
for col in data.columns[:-1]:
    temp = pd.crosstab(data['play'], data[col])
    tabs.append(temp)

In [96]:
pd.crosstab(data['play'], data['outlook'])

outlook,overcast,rainy,sunny
play,,,
no,0,2,3
yes,4,3,2


In [117]:
pd.concat(tabs, axis=1)

,overcast,rainy,sunny,cool,hot,mild,high,normal,False,True
play,,,,,,,,,,
no,0,2,3,1,2,2,4,1,2,3
yes,4,3,2,3,2,4,3,6,6,3


In [120]:
freqy = data.play.value_counts()[::-1]
freqy

no     5
yes    9
Name: play, dtype: int64

In [122]:
Xprob = pd.concat(tabs, axis=1)/freqy.to_frame().values
Xprob

,overcast,rainy,sunny,cool,hot,mild,high,normal,False,True
play,,,,,,,,,,
no,0.000000,0.400000,0.600000,0.200000,0.400000,0.400000,0.800000,0.200000,0.400000,0.600000
yes,0.444444,0.333333,0.222222,0.333333,0.222222,0.444444,0.333333,0.666667,0.666667,0.333333


In [132]:
freqy.values.reshape(-1,1)

array([[5],
       [9]], dtype=int64)

In [144]:
yprob = data.play.value_counts(normalize=True)[::-1]   ## 모든 데이터 포인트가 동일한 정도의 스케일(중요도)로 반영되도록 해주는 게 정규화(Normalization)
yprob

no     0.357143
yes    0.642857
Name: play, dtype: float64

In [137]:
data.play.value_counts()[::-1]

no     5
yes    9
Name: play, dtype: int64

#### 우리가 구하려는 것은 날씨가 Sunny, Mild, High, True일 때, 골프를 칠꺼냐(Yes) 말꺼냐(No)이다. 따라서:

- P(Sunny, Mild, High, True|Yes)과
- P(Sunny, Mild, High, True|No) 비교해서,
#### 큰 값으로 선택하면 된다.

#### 위에서 언급 했듯이, 나이브 베이지안 분류기는 모든 변수들의 사건이 독립이라 가정하기 때문에,

- P(Sunny, Mild, High, True|Yes)는
- P(Sunny|Yes) × ...P(True|Yes)과 같다.
#### 위의 표를 가지고 직접 계산하여 보자. 먼저 아래와 같은 값을 먼저 계산한다.

- Xprob = [ ⋯,P(xi|y),⋯ ]
- yprob =  P(y)

In [142]:

Xprob['rainy']*Xprob['sunny']*Xprob['cool']*Xprob['hot']*Xprob['mild']

play
no     0.007680
yes    0.002439
dtype: float64

In [154]:
yprob.index[(Xprob['rainy']*Xprob['sunny']*Xprob['cool']*Xprob['hot']*Xprob['mild']).argmax()]

'no'

In [158]:
prob = yprob.copy()
for col in ["sunny", "mild", "high", "True"]:
    prob *= Xprob[col].values
    
print(yprob.index[prob.argmax()])
prob

no


no     0.041143
yes    0.007055
Name: play, dtype: float64

In [156]:
logprob = np.log(yprob)
for col in ["sunny", "mild", "high", "True"]:
    logprob += np.log(Xprob[col].values)
    
print(yprob.index[logprob.argmax()])
logprob

no


no    -3.190705
yes   -4.954065
Name: play, dtype: float64